In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline
from IPython.display import set_matplotlib_formats
set_matplotlib_formats('png', 'pdf')

/tmp/ipykernel_1586/1799943160.py:3: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats('png', 'pdf')


# Exercise 8

<img src='./images/08.png' width=800>

In [ ]:
import os
import mlflow
os.environ['MLFLOW_TRACKING_URI'] = './mlruns'
mlflow.set_tracking_uri(os.environ.get('MLFLOW_TRACKING_URI'))

In [ ]:
import mlflow
mlflow.set_experiment('Exercise_8')

2025/04/08 14:06:53 INFO mlflow.tracking.fluent: Experiment with name 'Exercise_8' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///home/spakdel/my_projects/Books/Inside-Deep-Learning/Exercises_InsideDeepLearning/Chapter_02/mlruns/476357225522364280', creation_time=1744108613358, experiment_id='476357225522364280', last_update_time=1744108613358, lifecycle_stage='active', name='Exercise_8', tags={}>

In [ ]:
from sklearn.datasets import make_moons
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import optuna
import torch.nn as nn
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from utils import (train_network, accuracy_score_wrapper, 
                f1_score_wrapper, roc_auc_score_micro_wrapper, 
                weight_reset, set_seed)
from torchinfo import summary
from mlflow import MlflowClient
from mlflow.types import Schema, TensorSpec
from mlflow.models import ModelSignature


random_state = 42
set_seed(random_state)

X_train, Y_train = make_moons(n_samples=8000, noise=0.4, random_state=random_state)
X_valid, Y_valid = make_moons(n_samples=200, noise=0.4, random_state=random_state)

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                            torch.tensor(Y_train, dtype=torch.long))
valid_dataset = TensorDataset(torch.tensor(X_valid, dtype=torch.float32),
                            torch.tensor(Y_valid, dtype=torch.long))

batch_sizes = [2 ** i for i in range(1, 12)]

def plot_results(data_df, close=True):
    sns.lineplot(data_df, x='epoch', y='valid AUC', label='valid AUC')
    plt.xlabel('epoch')
    plt.ylabel('valid AUC')
    plt.title('valid AUC')
    fig = plt.gcf()
    if close:
        plt.close()
    return fig

def plot_results_batch_sizes(data_list, close=True):
    sns.lineplot(x=batch_sizes, y=data_list)
    plt.xlabel('batch sizes')
    plt.ylabel('valid Acc')
    plt.title('valid Acc per batch size')
    # plt.xticks(batch_sizes)
    fig = plt.gcf()
    if close:
        plt.close()
    return fig

def plot_time_batch_sizes(data_list, close=True):
    sns.lineplot(x=batch_sizes, y=data_list)
    plt.xlabel('batch sizes')
    plt.ylabel('total time')
    plt.title('total time per batch size')
    # plt.xticks(batch_sizes)
    fig = plt.gcf()
    if close:
        plt.close()
    return fig

epochs = 20
in_features = 2
out_features = 2
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
params = {'device': device,
        'epochs': epochs}
loss_func = nn.CrossEntropyLoss()


model = nn.Sequential(
    nn.Linear(2,  30),
    nn.Tanh(),
    nn.Linear(30,  30),
    nn.Tanh(),
    nn.Linear(30, 2),
)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)
params['optimizer'] = optimizer.defaults
results_batch_sizes = []
time_batch_sizes = []
for batch_size in batch_sizes:
    print(f'trainig: {batch_size} batch sizes')
    params['batch_size'] = batch_size
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)
    with mlflow.start_run(nested=True, run_name=f'batch_size_{batch_size}'):
        mlflow.log_params(params)
        
        with open ("model_summary.txt", "w") as f:
            f.write(str(summary(model)))
        mlflow.log_artifact("model_summary.txt")
        # model.apply(weight_reset)
        fc_results = train_network(
            model=model,
            loss_func=loss_func,
            train_loader=train_dataloader,
            valid_loader=valid_dataloader,
            epochs=epochs,
            optimizer=optimizer,
            score_funcs={'Acc':accuracy_score_wrapper, 'F1':f1_score_wrapper, 'AUC':roc_auc_score_micro_wrapper },
            device=device,
            # checkpoint_file_save='model.pth',
            # checkpoint_every_x=10,
        )
        
        # input_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 2))])
        # output_schema = Schema([TensorSpec(np.dtype(np.float32), (-1, 2))])
        # signature = ModelSignature(inputs=input_schema, outputs=output_schema)
        # mlflow.pytorch.log_model(model, "model", signature=signature)
        mlflow.log_figure(plot_results(fc_results), "valid_AUC.png")
    results_batch_sizes.append(fc_results['valid Acc'].iloc[-1])
    time_batch_sizes.append(fc_results['total time'].iloc[-1])
mlflow.log_figure(plot_results_batch_sizes(results_batch_sizes), 'valid_acc_per_batch_sizes.png')
mlflow.log_figure(plot_time_batch_sizes(time_batch_sizes), 'time_per_batch_sizes.png')


/home/spakdel/miniconda3/envs/python3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


trainig: 2 batch sizes


Epoch: 100%|██████████| 20/20 [01:07<00:00,  3.38s/it]


trainig: 4 batch sizes


Epoch: 100%|██████████| 20/20 [00:36<00:00,  1.84s/it]


...

Epoch: 100%|██████████| 20/20 [00:03<00:00,  6.37it/s]


trainig: 1024 batch sizes


Epoch: 100%|██████████| 20/20 [00:03<00:00,  5.10it/s]


trainig: 2048 batch sizes


Epoch: 100%|██████████| 20/20 [00:03<00:00,  6.38it/s]


<img src='./images/time_per_batch_sizes.png' width=800>

<img src='./images/valid_acc_per_batch_sizes.png' width=800>

<img src='./images/valid_auc_batch_size.png' width=800>